# HST WFC3 Create Negative and Root-Mean-Square Images

**Author:** Mitchell Revalski

**Updated:** July 28, 2021

***

## Introduction

This Jupyter Notebook was created by Dr. Mitchell Revalski to generate negative (NEG) and root-mean-square (RMS) images from user-supplied Hubble Space Telescope (HST) Wide Field Camera 3 (WFC3) science (SCI) and inverse variance weight map (WHT) drizzles that were produced with [DrizzlePac's](https://www.stsci.edu/scientific-community/software/drizzlepac.html) [AstroDrizzle](https://drizzlepac.readthedocs.io/en/latest/astrodrizzle.html) software. These images are useful for determining detection thresholds and estimating noise properties when measuring the photometry of objects in the SCI image with [Source Extractor](https://www.astromatic.net/software/sextractor/) and similar codes. The code is specifically designed to use WHT maps that were generated with `final_wht_type='IVM'`.

Please send questions, comments, and suggestions to [Mitchell Revalski](https://www.mitchellrevalski.com). Thank you, and have a nice day!

***

## Software

The Python environment used to run this notebook was built using conda:<br>
<code>
(base) ...:...$ conda create -n ac3 stsci
</code>

The jupyter notebook can then be run using the following call sequence:<br>
<code>
(base) ...:...$ conda activate ac3
(ac3)  ...:...$ jupyter notebook
</code>

The critical package versions that the code was tested with include:<br>
<code>
python -> 3.7.4
numpy -> 1.17.2
astropy -> 3.2.1
</code>    

***

## Table of Contents <a class="anchor" id="tag0"><a>


**&emsp; 1) [Setup Options](#setup)<br>**
**&emsp; 2) [Define Function](#def)<br>**
**&emsp; 3) [Create Images](#create)<br>**
    
***
    
## Imports

The following packages are required to run the Jupyter Notebook:
 - *os* - change and make directories
 - *sys* - set paths to external codes
 - *numpy* - math and array functions
 - *astropy.io.fits* - import FITS files
 - *drizzlepac.pixreplace* - replace NaN's and Inf's 

In [ ]:
import os
import sys
import numpy
from astropy.io import fits
from drizzlepac import pixreplace

## Setup Options  <a class="anchor" id="setup"><a>
[Table of Contents](#tag0)

In [ ]:
# Set the working data directory where the SCI and WHT files are located.
data_dir = '/Users/../../'

# Set the location of create_neg_rms_img.py if the code is in another directory.
sys.path.append('/Users/../../')

# Audibly announce when the code has finished running.
speak = True

'''
The following options control how the code will create and modify the files.
Please see the option descriptions in the definition of create_neg_rms_img()
below to choose the best settings for your SCI, WHT, NEG, and RMS images.
'''

sci_img = 'your_drizzle_filename_drz_sci.fits'

wht_img = 'your_drizzle_filename_drz_wht.fits'

neg_out = 'your_drizzle_filename_drz_neg.fits'

rms_out = 'your_drizzle_filename_drz_rms.fits'

fix_nan = True

fix_inf = True

set_img = False

## Define Function<a class="anchor" id="def"><a>
[Table of Contents](#tag0)

In [ ]:
def create_neg_rms_img(sci_img, wht_img, neg_out, rms_out, fix_nan, fix_inf, set_img):

    '''
    \n
    \nWelcome to create_neg_rms_img!

    This function is designed to take user
    provided science and weight FITS images,
    calculate negative and root mean square
    results, and save them as new FITS files.

    The function parameters are:

    sci_img: filename of your input science image
    wht_img: filename of your input weight image
    neg_out: desired filename of your negative image
    rms_out: desired filename of your rms image
    fix_nan: replace NaNs with zero (True or False)
    fix_inf: replace Infs with 1e10 (True or False)
    set_img: create SExtractor images (True or False)

    Notes:
     - All filenames should be entered as strings.
     - The code assumes data are in extension [0].
     - pixreplace may freeze for large files when
     there is insufficient memory and the function
     may be run in a new jupyter kernel or terminal.

    Written by Mitchell Revalski
    Last Updated on February 18, 2021
    Contributions by Laura Prichard and Marc Rafelski
    '''

    # History:
    # Created on June 9, 2020
    # Updated on July 13, 2020
    # - added commands to close the fits files.
    # - added NaN+Inf replacement for SCI and WHT.
    # Updated on October 27, 2020
    # - added set_img for Source Extractor compatibility.
    # - removes the problematic headlets but also the WCS.
    # Updated on February 18, 2021
    # - modified writeto in set_img to save WCS header.
    # - corrected minor typographical errors in notes.

    print('Calling create_neg_rms_img()...')

    # Import packages
    import numpy
    from astropy.io import fits
    from drizzlepac import pixreplace

    # Open the SCI image
    sci_hdul = fits.open(sci_img)
    sci_data = sci_hdul[0].data

    # Open the WHT image
    wht_hdul = fits.open(wht_img)
    wht_data = wht_hdul[0].data

    # Generate the NEG image
    print('\nCalculating negative image...')
    neg_img = (-1.0*sci_data)

    # Generate the RMS image
    print('\nCalculating rms image...\n')
    rms_img = (1.0/numpy.sqrt(wht_data))

    # Replace the data arrays
    sci_hdul[0].data = neg_img
    wht_hdul[0].data = rms_img

    # Export the images
    sci_hdul.writeto(neg_out, overwrite=True)
    wht_hdul.writeto(rms_out, overwrite=True)

    # Replace NaNs if selected
    if fix_nan == True:
        pixreplace.replace(sci_img)
        pixreplace.replace(wht_img)
        pixreplace.replace(neg_out)
        pixreplace.replace(rms_out)

    # Replace Infs if selected
    if fix_inf == True:
        pixreplace.replace(sci_img, pixvalue=numpy.inf, newvalue=1.0e10)
        pixreplace.replace(wht_img, pixvalue=numpy.inf, newvalue=1.0e10)
        pixreplace.replace(neg_out, pixvalue=numpy.inf, newvalue=1.0e10)
        pixreplace.replace(rms_out, pixvalue=numpy.inf, newvalue=1.0e10)

    # Close the images
    sci_hdul.close()
    wht_hdul.close()

    # Create Source Extractor images
    if set_img == True:

        # Open the images
        sci_hdul = fits.open(sci_img)
        wht_hdul = fits.open(wht_img)
        neg_hdul = fits.open(neg_out)
        rms_hdul = fits.open(rms_out)

        # Write only the first extension for Source Extractor
        fits.writeto(sci_img, sci_hdul[0].data, sci_hdul[0].header, overwrite=True)
        fits.writeto(wht_img, wht_hdul[0].data, wht_hdul[0].header, overwrite=True)
        fits.writeto(neg_out, neg_hdul[0].data, neg_hdul[0].header, overwrite=True)
        fits.writeto(rms_out, rms_hdul[0].data, rms_hdul[0].header, overwrite=True)

        # Close the images
        sci_hdul.close()
        wht_hdul.close()
        neg_hdul.close()
        rms_hdul.close()

    # Print completion
    print('\nDone.')

# End of function

## Create Images <a class="anchor" id="create"><a>
[Table of Contents](#tag0)

In [ ]:
# Call the function.

os.chdir(data_dir)

create_neg_rms_img(sci_img = sci_img, 
                   wht_img = wht_img, 
                   neg_out = neg_out, 
                   rms_out = rms_out, 
                   fix_nan = fix_nan, 
                   fix_inf = fix_inf, 
                   set_img = set_img)

if speak == True:
    
    os.system("say 'your notebook has finished running.'")

[Table of Contents](#tag0)